In [29]:
import pandas as pd, numpy as np, requests, logging

class DataEnedisAdeme:
    # enedis records endpoint has limitations ~ default 10 if not set  # combier limit et offset pour imiter la pagination limite à 10_000 lignes 
    # while export endpoint has no limitations of rows
    def __init__(self, path_enedis_csv=None):
        self.path_enedis_csv = path_enedis_csv
        self.debugger = {}

    def load_enedis_from_csv(self):
        res = pd.read_csv(self.path_enedis_csv, sep=';')
        res = res.rename(columns={'Adresse': 'adresse', 'Nom Commune': 'nom_commune', 'Code Commune': 'code_commune'})
        return res

    def load_get_data_pandas(self, url):
        """Extract pandas dataframe from any valid url."""
        res = requests.get(url).json().get('results')
        return pd.DataFrame(res)

    def get_url_enedis_year(self, annee): 
        """Génerer l'url pour requeter l'api enedis. filtres sur le nbre de lignes.
        param: annee : int année souhaitée
        """
        return f"https://data.enedis.fr/api/explore/v2.1/catalog/datasets/consommation-annuelle-residentielle-par-adresse/records?where=annee%20%3D%20date'{annee}'"

    def get_url_enedis_year_rows(self, annee, rows): 
        """Génerer l'url pour requeter l'api enedis. filtres sur l'année et le nbre de lignes.
        param: annee : int année souhaitée
               rows : nbre de lignes souhaitées.
        """
        return f"https://data.enedis.fr/api/explore/v2.1/catalog/datasets/consommation-annuelle-residentielle-par-adresse/records?where=annee%20%3D%20date'{annee}'&limit={rows}"

    def get_url_ademe_filter_on_ban(self, key):
        """Générer l'url pour requeter l'api de l'ademe (dpe depuis juillet 2021)."""
        return f"https://data.ademe.fr/data-fair/api/v1/datasets/dpe-v2-logements-existants/lines?size=1000&format=json&qs=Identifiant__BAN%3A{key}"

    def get_ban_res(self, addr):
        """requeter l'api de ban de facon unitaire càd une seule adresse en entrée.
           produit une seule sortie avec les infos sur l'adresse si elle existe (dont l'id BAN).
           Si l'adresse n'existe pas renvoie None.
        """
        ADDOK_URL = 'http://api-adresse.data.gouv.fr/search/'

        params = {
            'q': addr,
            'limit': 1
        }
        response = requests.get(ADDOK_URL, params=params)
        if response.status_code == 200:
            j = response.json()
            if len(j.get('features')) > 0:
                first_result = j.get('features')[0]
                lon, lat = first_result.get('geometry').get('coordinates')
                first_result_all_infos = { **first_result.get('properties'), **{"lon": lon, "lat": lat}, **{'full_adress': addr}}
                return first_result_all_infos
            else:
                return
        else:
            return

    def get_enedis_with_ban_pandas(self, requete_url_enedis, from_export): 
        """obtenir le df pandas des données enedis (requete) + les données de la BAN pour les adresses trouvées."""
        
        if not from_export:
            # 1- requeter enedis
            logging.warning(f"Extract from url enedis :\n {requete_url_enedis}")
            self.debugger.update({'source_enedis': requete_url_enedis})

            enedis_data = self.load_get_data_pandas(requete_url_enedis)
        else:
            # 1bis- lire depuis csv enedis
            logging.warning(f"Loading from :\n {self.path_enedis_csv}")
            self.debugger.update({'source_enedis': self.path_enedis_csv})

            enedis_data = self.load_enedis_from_csv()
            enedis_data = enedis_data[enedis_data['Code Département']==75]
        self.debugger.update({'sample_enedis_data': enedis_data.tail(5)})

        # 2- extraire les adresses complètes
        enedis_adresses_list = list(zip(enedis_data.adresse.values.tolist(),
                                        enedis_data.code_commune.values.tolist(),
                                        enedis_data.nom_commune.values.tolist()))
        enedis_adresses_list = [f"{a} {b} {c}" for a,b,c in enedis_adresses_list]
        enedis_data['full_adress'] = enedis_adresses_list
        self.debugger.update({'enedis_full_adresses': set(enedis_adresses_list)})

        # 3- requeter l'api de la BAN sur les adresses enedis
        # ban_data = pd.DataFrame([self.get_ban_res(_) for _ in enedis_adresses_list]) # drop nones 
        # ban_data = [self.get_ban_res(_) for _ in enedis_adresses_list]
        sample_enedis = enedis_adresses_list #[0:2005]
        ban_data = [self.get_ban_res(_) for _ in sample_enedis]
        ban_data = [_ for _ in ban_data if str(_).lower() != 'none']
        logging.warning(f"Valid data BAN : {len(ban_data)}/{len(sample_enedis)}")
        ban_data = pd.DataFrame(ban_data)
        vectorized_upper = np.vectorize(str.upper, cache=True)
        ban_data['label'] = vectorized_upper(ban_data['label'].values)
        self.debugger.update({'sample_ban_data': ban_data.tail(5)})

        # 4- renvoyer le resultat mergé : si une adresse est pas  trouvée on tej la data enedis (cf. inner join)
        return pd.merge(enedis_data, ban_data, how='inner', left_on='full_adress', right_on='full_adress').rename(columns={'id': 'id_BAN'})

    def get_enedis_with_ban_with_ademe(self, requete_url_enedis, from_export:bool=False):
        """extraire la data enedis x ban x ademe au complet à partir d'un dataframe enedis."""
        
        # extraire enedis avec les infos de la BAN
        enedis_with_ban_data = self.get_enedis_with_ban_pandas(requete_url_enedis, from_export=from_export)
        # sur la base des Identifiants BAN de enedis, aller chercher les logements mappés sur ces codes BAN
        # plusieurs adresses possibles pour un id BAN 
        # car les données enedis sont regroupées/agrégées. Toutefois, on dispose de la moyenne.
        ademe_data = []
        ademe_data_res = [requests.get(self.get_url_ademe_filter_on_ban(_)).json().get('results') for _ in enedis_with_ban_data.id_BAN.values.tolist()] # liste à 2 niveaux # pour chaque Id_BAN on a plusieurs lignes ademe
        for _ in ademe_data_res:
            ademe_data.extend(_)
        ademe_data = pd.DataFrame(ademe_data)

        ademe_data = ademe_data.add_suffix('_ademe')
        enedis_with_ban_data = enedis_with_ban_data.add_suffix('_enedis_with_ban')

        del ademe_data_res
        return pd.merge(ademe_data,
                        enedis_with_ban_data,
                        how='left',
                        left_on='Identifiant__BAN_ademe',
                        right_on='id_BAN_enedis_with_ban')

    def extract_year_rows(self, year:int=2018, rows:int=20):
        res = self.get_enedis_with_ban_with_ademe(self.get_url_enedis_year_rows(year,rows))
        logging.warning(f"Extraction results : {res.shape[0]} rows, {res.shape[1]} columns.")
        return res
    
    def extract(self):
        res = self.get_enedis_with_ban_with_ademe(None, from_export=True)
        logging.warning(f"Extraction results : {res.shape[0]} rows, {res.shape[1]} columns.")
        self.result = res
        return res.head(3)

In [5]:
# avec ceci on voit que la plupart des adresses enedis sont des logements 'residentiels' avec au moins 10 logements et au plus +de 1000 logements
tmp_url = "https://data.enedis.fr/api/explore/v2.1/catalog/datasets/consommation-annuelle-residentielle-par-adresse/records?select=segment_de_client%2C%20annee%2C%20count\
    (code_iris)%20as%20nb%2C%20min(nombre_de_logements)%20as%20nb_min_logements%2C%20max(nombre_de_logements)%20as%20nb_max_logements&group_by=segment_de_client%2C%20annee&limit=20"

"""

"metas": {
    "default": {
        "title": "Consommation d'électricité annuelle résidentielle par adresse",
        "description": "<p>Le jeu de données présente la consommation électrique annuelle des adresses de 10 logements ou plus. Un logement correspond à un point de livraison (ou PDL) de type résidentiel.\n<br><br>Les données sont mises à disposition à titre purement indicatif sans garantie quant au degré de fiabilité des adresses. Enedis ne saurait, par conséquent, être tenue responsable en cas de défaut de fiabilité.\n<br><br>À partir de 2021 et suite à un changement dans le référentiel des adresses, la normalisation de celles-ci peut faire apparaitre des écarts avec les années précédentes.</p>\n<p>Visualisez aussi les données d'Enedis grâce à nos datavisualisations sur notre <a href=\"https://data.enedis.fr\" target=\"_blank\">site internet</a>.</p>\n<p>Une question sur ces données ? N'hésitez pas à utiliser notre <a href=\"https://data.enedis.fr/page/contact/\" target=\"_blank\">formulaire de contact</a>.</p>\n\n",
        "theme": [
            "Consommation"
"""

DataEnedisAdeme().load_get_data_pandas(tmp_url)


,segment_de_client,annee,nb,nb_min_logements,nb_max_logements
0,RESIDENTIEL,2018-01-01T00:00:00+00:00,391099,10,1189
1,RESIDENTIEL,2019-01-01T00:00:00+00:00,399791,10,1124
2,RESIDENTIEL,2020-01-01T00:00:00+00:00,405605,10,1435
3,RESIDENTIEL,2021-01-01T00:00:00+00:00,407053,10,1189
4,RESIDENTIEL,2022-01-01T00:00:00+00:00,394100,10,1190
5,RESIDENTIEL,2023-01-01T00:00:00+00:00,397398,10,1196


In [6]:
tool = DataEnedisAdeme()
exple = tool.extract_year_rows(2018, 10) # entre -1 et 100
# exple = tool.extract_year(2018)

 https://data.enedis.fr/api/explore/v2.1/catalog/datasets/consommation-annuelle-residentielle-par-adresse/records?where=annee%20%3D%20date'2018'&limit=10


In [32]:
# exple
tool.debugger.get('sample_enedis_data')

,annee,code_iris,nom_iris,numero_de_voie,indice_de_repetition,type_de_voie,libelle_de_voie,code_commune,nom_commune,segment_de_client,nombre_de_logements,consommation_annuelle_totale_de_l_adresse_mwh,consommation_annuelle_moyenne_par_site_de_l_adresse_mwh,consommation_annuelle_moyenne_de_la_commune_mwh,adresse,code_epci,code_departement,code_region,tri_des_adresses
95,2018,740100603,Les Fins,3,None,RUE,DE LA FRATERNITE,74010,ANNECY,RESIDENTIEL,20,90.852,4.543,3.092,3 RUE DE LA FRATERNITE,200066793,74,84,410716
96,2018,740100205,Gare,17,None,RUE,DE LA GARE,74010,ANNECY,RESIDENTIEL,55,79.587,1.447,3.092,17 RUE DE LA GARE,200066793,74,84,410737
97,2018,740100205,Gare,5,None,RUE,DE LA GARE,74010,ANNECY,RESIDENTIEL,16,50.308,3.144,3.092,5 RUE DE LA GARE,200066793,74,84,410742
98,2018,74010xxxx,Fictif - Annecy,1,None,RUE,DE LA JONCHERE,74010,ANNECY,RESIDENTIEL,17,39.510,2.324,3.092,1 RUE DE LA JONCHERE,200066793,74,84,410756
99,2018,74010xxxx,Fictif - Annecy,5,None,RUE,DE LA LATHARDAZ,74010,ANNECY,RESIDENTIEL,14,15.958,1.140,3.092,5 RUE DE LA LATHARDAZ,200066793,74,84,410762


In [ ]:
tool = DataEnedisAdeme('../ressources/data/enedis_2018.csv')
tool.extract()

# avec un sample de 1005 lignes 
# WARNING:root:Loading from :
# ../ressources/data/enedis_2018.csv
# WARNING:root:Valid data BAN : 1004/1005
# WARNING:root:Extraction results : 7574 rows, 280 columns. => 4min

# -----------------------
# scope de cette extraction : PARIS 2018 code département = 75 et année = 2018
# 50k lignes de enedis => 50k adresses BAN mappées
# en croisant avec ademe : on a 465_961 lignes asachant q'une adresse enedis regroupe pluisuers logements ademe
# ---> extract sauvegardé 

 ../ressources/data/enedis_2018.csv


,Conso_chauffage_dépensier_é_finale_ademe,Volume_stockage_générateur_ECS_n°1_ademe,Conso_é_finale_installation_ECS_ademe,Nom__commune_(BAN)_ademe,Emission_GES_chauffage_ademe,Conso_ECS_é_finale_énergie_n°2_ademe,Conso_ECS_é_finale_énergie_n°1_ademe,Besoin_refroidissement_ademe,Conso_chauffage_dépensier_installation_chauffage_n°1_ademe,Coût_total_5_usages_ademe,...,y_enedis_with_ban,city_enedis_with_ban,district_enedis_with_ban,context_enedis_with_ban,type_enedis_with_ban,importance_enedis_with_ban,street_enedis_with_ban,lon_enedis_with_ban,lat_enedis_with_ban,_type_enedis_with_ban
0,16170.8,0.0,1548.3,Paris,2987.6,0.0,1548.3,0,16170.8,1083.0,...,6866570.89,Paris,Paris 17e Arrondissement,"75, Paris, Île-de-France",housenumber,0.66547,Passage Flourens,2.327087,48.897252,NaN
1,20081.5,0.0,5617.0,Paris,3737.2,0.0,5617.0,0,20081.5,2150.5,...,6866570.89,Paris,Paris 17e Arrondissement,"75, Paris, Île-de-France",housenumber,0.66547,Passage Flourens,2.327087,48.897252,NaN
2,38501.7,0.0,5812.1,Paris,10285.3,0.0,5812.1,0,38501.7,3529.6,...,6863733.09,Paris,Paris 16e Arrondissement,"75, Paris, Île-de-France",housenumber,0.77255,Avenue Foch,2.278373,48.871447,address


In [32]:
tool.result.to_csv('../ressources/data/enedis_ban_ademe_extract_PARIS_2018_400k_rows_250_min.csv')

In [35]:
tool.result.to_parquet('../ressources/data/enedis_ban_ademe_extract_PARIS_2018_400k_rows_250_min.parquet', index=False, engine='pyarrow')

In [37]:
pd.read_csv('../ressources/data/enedis_ban_ademe_extract_PARIS_2018_400k_rows_250_min.csv',
            low_memory=True).head(3)

C:\Users\gbeno\AppData\Local\Temp\ipykernel_22268\374963924.py:1: DtypeWarning: Columns (8,45,163,169,171,172,176,177,180,181,182,185,186,188,191,192,193,194,195,196,198,199,202,206,207,212,218,227,228,229,232,235,236,237,238,242,247,265,280) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv('../ressources/data/enedis_ban_ademe_extract_PARIS_2018_400k_rows_250_min.csv',


,Unnamed: 0,Conso_chauffage_dépensier_é_finale_ademe,Volume_stockage_générateur_ECS_n°1_ademe,Conso_é_finale_installation_ECS_ademe,Nom__commune_(BAN)_ademe,Emission_GES_chauffage_ademe,Conso_ECS_é_finale_énergie_n°2_ademe,Conso_ECS_é_finale_énergie_n°1_ademe,Besoin_refroidissement_ademe,Conso_chauffage_dépensier_installation_chauffage_n°1_ademe,...,y_enedis_with_ban,city_enedis_with_ban,district_enedis_with_ban,context_enedis_with_ban,type_enedis_with_ban,importance_enedis_with_ban,street_enedis_with_ban,lon_enedis_with_ban,lat_enedis_with_ban,_type_enedis_with_ban
0,0,16170.8,0.0,1548.3,Paris,2987.6,0.0,1548.3,0,16170.8,...,6866570.89,Paris,Paris 17e Arrondissement,"75, Paris, Île-de-France",housenumber,0.66547,Passage Flourens,2.327087,48.897252,NaN
1,1,20081.5,0.0,5617.0,Paris,3737.2,0.0,5617.0,0,20081.5,...,6866570.89,Paris,Paris 17e Arrondissement,"75, Paris, Île-de-France",housenumber,0.66547,Passage Flourens,2.327087,48.897252,NaN
2,2,38501.7,0.0,5812.1,Paris,10285.3,0.0,5812.1,0,38501.7,...,6863733.09,Paris,Paris 16e Arrondissement,"75, Paris, Île-de-France",housenumber,0.77255,Avenue Foch,2.278373,48.871447,address


#### Autres

In [26]:
d = tool.load_enedis_from_csv()
d[d['Code Département']==75].shape

(50061, 19)

In [70]:
ADDOK_URL = 'http://api-adresse.data.gouv.fr/search/'

params = {
    'q': '8 RUE DE LA PLAINE 95355 ARTHIEUL',
    'limit': 1
}
response = requests.get(ADDOK_URL, params=params)
type(response.status_code)

int

In [89]:
# len(tool.debugger.get('enedis_full_adresses')) * 4 / 10
# 156236.4/3600 # env 43h

43.399

In [90]:
# [tool.get_ban_res(_) for _ in list(tool.debugger.get('enedis_full_adresses'))[0:10]]
tmp_adresses = tool.debugger.get('enedis_full_adresses')

In [94]:
data = tool.load_enedis_from_csv()
enedis_adresses_list = list(zip(data.adresse.values.tolist(),
                                        data.code_commune.values.tolist(),
                                        data.nom_commune.values.tolist()))
enedis_adresses_list = [f"{a} {b} {c}" for a,b,c in enedis_adresses_list]
data['full_adress'] = enedis_adresses_list
data.to_csv('../ressources/data/enedis_2018_full_adresses.csv')

In [36]:
tool.get_ban_res('10 PLACE HENRI FARMAN 16015 ANGOULEME')

{'label': '10 Place Henri Farman 16000 Angoulême',
 'score': 0.7444476923076923,
 'housenumber': '10',
 'id': '16015_1995_00010',
 'name': '10 Place Henri Farman',
 'postcode': '16000',
 'citycode': '16015',
 'x': 476170.96,
 'y': 6509602.91,
 'city': 'Angoulême',
 'context': '16, Charente, Nouvelle-Aquitaine',
 'type': 'housenumber',
 'importance': 0.57354,
 'street': 'Place Henri Farman',
 'lon': 0.125568,
 'lat': 45.649301,
 'full_adress': '10 PLACE HENRI FARMAN 16015 ANGOULEME'}

In [1]:
## test lecture parquet gzip

import pandas as pd
df = pd.read_parquet('../ressources/data/enedis_ban_ademe_extract_PARIS_2018_400k_rows_250_min.parquet')
df.to_parquet('../ressources/data/enedis_ban_ademe_extract_PARIS_2018_400k_rows_250_min_comp.parquet', engine='pyarrow', compression='gzip')

In [9]:
df = pd.read_parquet('../ressources/data/2_intermediary/enedis_ban_ademe_extract_PARIS_2018_400k_rows_250_min_comp.parquet', engine='pyarrow')
df.head()

,Conso_chauffage_dépensier_é_finale_ademe,Volume_stockage_générateur_ECS_n°1_ademe,Conso_é_finale_installation_ECS_ademe,Nom__commune_(BAN)_ademe,Emission_GES_chauffage_ademe,Conso_ECS_é_finale_énergie_n°2_ademe,Conso_ECS_é_finale_énergie_n°1_ademe,Besoin_refroidissement_ademe,Conso_chauffage_dépensier_installation_chauffage_n°1_ademe,Coût_total_5_usages_ademe,...,y_enedis_with_ban,city_enedis_with_ban,district_enedis_with_ban,context_enedis_with_ban,type_enedis_with_ban,importance_enedis_with_ban,street_enedis_with_ban,lon_enedis_with_ban,lat_enedis_with_ban,_type_enedis_with_ban
0,16170.8,0.0,1548.3,Paris,2987.6,0.0,1548.3,0,16170.8,1083.0,...,6866570.89,Paris,Paris 17e Arrondissement,"75, Paris, Île-de-France",housenumber,0.66547,Passage Flourens,2.327087,48.897252,None
1,20081.5,0.0,5617.0,Paris,3737.2,0.0,5617.0,0,20081.5,2150.5,...,6866570.89,Paris,Paris 17e Arrondissement,"75, Paris, Île-de-France",housenumber,0.66547,Passage Flourens,2.327087,48.897252,None
2,38501.7,0.0,5812.1,Paris,10285.3,0.0,5812.1,0,38501.7,3529.6,...,6863733.09,Paris,Paris 16e Arrondissement,"75, Paris, Île-de-France",housenumber,0.77255,Avenue Foch,2.278373,48.871447,address
3,39468.7,0.0,6130.1,Paris,10508.7,0.0,6130.1,0,39468.7,3637.1,...,6863733.09,Paris,Paris 16e Arrondissement,"75, Paris, Île-de-France",housenumber,0.77255,Avenue Foch,2.278373,48.871447,address
4,15376.1,0.0,5752.4,Paris,2816.4,0.0,5752.4,0,15376.1,1186.5,...,6863783.96,Paris,Paris 16e Arrondissement,"75, Paris, Île-de-France",housenumber,0.77255,Avenue Foch,2.282777,48.871931,None
